In [ ]:
from numpy import array
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Dense
from tensorflow.keras.layers import Flatten, LSTM, Conv1D, RNN, SimpleRNN
from tensorflow.keras.layers import GlobalMaxPooling1D, Bidirectional 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Bidirectional
from sklearn import preprocessing
from tensorflow.keras import regularizers
from sklearn.metrics import mean_squared_error
from math import sqrt
import pandas as pd
import numpy as np
import re
from tensorflow.keras.utils import plot_model
from tensorflow.keras import regularizers
import matplotlib.pyplot as plt

from scipy.io import loadmat



# Read Data

In [ ]:
df = pd.read_excel('file.xlsx',Sheet='Sheet1')
data=np.array(df)
print(data.shape)

In [ ]:
df.head()

In [ ]:
phiprev = []
thetaprev = []
psiprev = []
for i in range(0,data.shape[0]):
    if i == 0:
        phiprev.append(0)
        thetaprev.append(0)
        psiprev.append(0)
    else:
        phiprev.append(data[i-1,9])
        thetaprev.append(data[i-1,10])
        psiprev.append(data[i-1,11])
phiprev = np.array(phiprev)
thetaprev = np.array(thetaprev) 
psiprev = np.array(psiprev) 

In [ ]:
dataX = np.column_stack((data[:,0:9],phiprev,thetaprev,psiprev))
dataY = data[:,9:12]

min_max_scaler = preprocessing.MinMaxScaler()
dataX = min_max_scaler.fit_transform(dataX)

In [ ]:
split = 10000
dataXt = dataX[:split]
dataYt = dataY[:split]
dataXv = dataX[split:]
dataYv = dataY[split:]

input_dim = dataXt.shape[1]

print("input_dim:", input_dim)
print("\nTraining Testing")
print(dataXt.shape[0],dataXv.shape[0])

In [ ]:
timestep = 2
Xphithetapsi = dataXt
Xphithetapsi = array(Xphithetapsi).reshape(split//timestep, timestep, input_dim)
Yphithetapsi = array(dataYt[1::timestep])
print("Xphithetapsi shape",Xphithetapsi.shape)
print("Yphithetapsi shape",Yphithetapsi.shape)

# Main Model Training

In [ ]:
# model desing
inputs = Input(name='inputs',shape=[timestep,input_dim])
layer = LSTM(50, return_sequences=True)(inputs)
layer = Dropout(0.25)(layer)
layer = LSTM(20)(layer)
outputs = Dense(3, activation='linear')(layer)

modelphithetapsi1 = Model(inputs=[inputs],outputs=[outputs])

# Compile
modelphithetapsi1.compile(optimizer='RMSprop', loss='mean_squared_error',metrics = ['accuracy'])
modelphithetapsi1.summary()

history = modelphithetapsi1.fit(Xphithetapsi, Yphithetapsi,
                       epochs=10,
                       verbose=0,
                       batch_size=50)

print('Model Training done')

In [ ]:
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['training'], loc = 'upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['training'], loc = 'upper left')
plt.show()

In [ ]:
# print the model learning rate
import tensorflow.keras.backend as K
print(K.eval(modelphithetapsi1.optimizer.lr))

# Prediction and Incremental learning 

In [ ]:
# new model for incremental learning
model = Model(inputs=[inputs],outputs=[outputs])
model.compile(optimizer='RMSprop', loss='mean_squared_error',metrics = ['accuracy'])

In [ ]:
#copy weights
Wsave = modelphithetapsi1.get_weights()
model.set_weights(Wsave)

In [ ]:
results = []
stepinit = 3000
araya = [stepinit, stepinit, stepinit, stepinit]
for j in range(0,array(araya).shape[0]):
    XtestA = []
    step = araya[j]
    pt = stepinit*j
    Xtestt = dataXv[pt:pt+step]
    Ytest = dataYv[pt:pt+step]
    Ytest = array(Ytest[1::timestep])
    Xtest = array(Xtestt[:,0:9])
    
    test_input = np.column_stack((Xtest,Xtestt[:,9:12]))
    XtestA = test_input

    ip = array(XtestA).reshape(step//timestep, timestep, input_dim)
    test_output = model.predict(ip)
    test_output = array(test_output)
    results.append(test_output)
    Xtest1 = ip
    history = model.fit(Xtest1, Ytest,
                            epochs=10,
                            verbose=0,
                            batch_size=50)
results1 = []
XtestA = []
step = (data.shape[0]-split)-stepinit*array(araya).shape[0]
pt = stepinit*array(araya).shape[0]
Xtestt = dataXv[pt:pt+step]
Ytest = dataYv[pt:pt+step]
Ytest = array(Ytest[1::timestep])
Xtest = array(Xtestt[:,0:9])

test_input = np.column_stack((Xtest,Xtestt[:,9:12]))
ip = array(test_input).reshape(step//timestep, timestep, input_dim)
test_output = model.predict(ip)
test_output = array(test_output)
results1.append(test_output)

results = np.vstack((results,results1))
print(results.shape)


# RMSE Plots

In [ ]:
Yt1 = dataYv
Yt = array(Yt1[1::timestep])
test_output = np.array(results)
test_output = test_output.reshape((Yt.shape[0],3))


from sklearn.metrics import mean_squared_error
from math import sqrt
rmsephi = sqrt(mean_squared_error(Yt[:,0], test_output[:,0]))
print("RMSE phi",rmsephi)

rmsetheta = sqrt(mean_squared_error(Yt[:,1], test_output[:,1]))
print("RMSE theta",rmsetheta)

rmsepsi = sqrt(mean_squared_error(Yt[:,2], test_output[:,2]))
print("RMSE phi",rmsepsi)

print(rmsephi,rmsetheta,rmsepsi)

plt.plot(Yt[:,0])
plt.plot(test_output[:,0])
plt.title('Phi/Roll')
plt.xlabel('samples')
plt.ylabel('Roll angle')
plt.legend(['Reference', 'Predicted'], loc = 'upper right')
plt.show()

plt.plot(Yt[:,1])
plt.plot(test_output[:,1])
plt.title('Theta/Pitch')
plt.xlabel('samples')
plt.ylabel('Pitch angle')
plt.legend(['Reference', 'Predicted'], loc = 'upper right')
plt.show()

plt.plot(Yt[:,2])
plt.plot(test_output[:,2])
plt.title('Psi/Yaw')
plt.xlabel('samples')
plt.ylabel('Yaw angle')
plt.legend(['Reference', 'Predicted'], loc = 'upper right')
plt.show()